In [ ]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));

In [ ]:
var scenario = "IRU";
var year = 2020;
var month = 12;
var reportingNode = "CH";
var dataNode = "DT1.1";
var targetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                   p.Year == year &&
                                                                                                   p.Month == month &&
                                                                                                   p.Scenario == scenario).ToArrayAsync()).Single().Id;
var defaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                    p.Year == year &&
                                                                                                    p.Month == month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
var args = new ImportArgs(dataNode, year, month, Periodicity.Quarterly, scenario,null);
var reportingNodeByPartitionAndScenario = await DataSource.Query<PartitionByReportingNode>().ToDictionaryAsync(x => (x.Id, x.Scenario), x => x.ReportingNode);
var reportingNodeByPartition = await DataSource.Query<PartitionByReportingNode>().ToDictionaryAsync(x => x.Id, x => x.ReportingNode);

reportingNodeByPartition


In [ ]:
await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario.csv");
await CleanDatabaseAsync<YieldCurve>();
var importLog = await Import.FromFile("../Files/800.Parameters/YieldCurveScenario.csv").WithType<YieldCurve>().WithTarget(DataSource).WithTarget(Workspace).ExecuteAsync();
Workspace.Query<YieldCurve>().First()

In [ ]:
var dataNodesByCurrencyAndReportingNode = Workspace.Query<DataNode>()
    .Select(x => new {SystemName = x.SystemName, 
        Currency = x.FunctionalCurrency, 
        ReportingNode = reportingNodeByPartition[x.Partition]})
        .GroupBy(x => new Tuple<string, string>(x.Currency, x.ReportingNode).ToValueTuple())
        .Select(x => new{ 
            Currency = x.Key.Item1, 
            ReportingNode = x.Key.Item2, 
            DataNodes = x.Select(y => y.SystemName).ToList()
            }).ToList();
    dataNodesByCurrencyAndReportingNode
    /*.Select(g => new {
        Key = g.Key,
        DataNodes = g.Select(dn => dn.SystemName).ToList()
    });
dataNodes
.GroupBy(i => i.A)
    .Select(g => new GroupItem() 
    { 
        A = g.Key, 
        AllBsWithinA = g.Select(i => i.B).ToList() 
    });*/

In [ ]:
var improvedQuery  = Workspace.Query<DataNode>()
        .GroupBy(x => new Tuple<string, Guid>(x.FunctionalCurrency, x.Partition).ToValueTuple())
        .Select(x => new{ 
            Currency = x.Key.Item1, 
            ReportingNode = reportingNodeByPartition[x.Key.Item2], 
            DataNodes = x.Select(y => y.SystemName).ToList()
            });
improvedQuery.ToList()
            

In [ ]:
var updatableDataNodesByPartition = await Workspace.Query<YieldCurve>()
    .Join(improvedQuery, 
    yc => yc.Currency, 
    dn => dn.Currency, 
    (yc, dn) => new { 
                    Args = new ImportArgs( dn.ReportingNode,
                                           yc.Year, 
                                           yc.Month,
                                           Periodicity.Quarterly,
                                           yc.Scenario,
                                           ImportFormats.Cashflow),
                    DataNodes = dn.DataNodes})
                    .OrderBy(x => x.Args.Year).ThenBy(x => x.Args.Month).ToArrayAsync();
                    updatableDataNodesByPartition[0].Args

In [ ]:
Activity.Start();
foreach (var dataNodesInPartition in updatableDataNodesByPartition){
        var args = dataNodesInPartition.Args;
        /*var partition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;*/
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new {ReportingNode = args.ReportingNode, Year= args.Year, Month = args.Month, Scenario = args.Scenario});
        var retrivedRawVariables = await DataSource.Query<RawVariable>().Where(x => dataNodesInPartition.DataNodes.Contains(x.DataNode)).ToArrayAsync();
        if(!retrivedRawVariables.Any()) 
            {
                await Workspace.UpdateAsync<RawVariable>(retrivedRawVariables);

                var storage = new ImportStorage(dataNodesInPartition.Args, DataSource, Workspace);
                await storage.InitializeAsync();
                var universe = Scopes.ForStorage(storage).ToScope<IModel>();
                var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
                var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
                if (await CompareCalculatedToDBAsync(args, ivs)){
                    ApplicationMessage.Log(Warning.VariablesAlreadyCalculated, "");
                    return Activity.Finish();
                }
                if(Activity.HasErrors()) return Activity.Finish();
    
                await Workspace.UpdateAsync<IfrsVariable>(ivs);
                await CommitToDatabase<IfrsVariable>(storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));  
                await CommitToDatabase<RawVariable>(storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));

            }
    }

    
Activity.Finish();

In [ ]:
/*await Workspace.Query<YieldCurve>()
               .Join(Workspace.Query<DataNode>(), 
                     yc => yc.Currency, 
                     dn => dn.FunctionalCurrency, 
                     (yc, dn) => new {
                                    Args = new ImportArgs( reportingNodeByPartition[dn.Partition],
                                                 yc.Year, 
                                                 yc.Month,
                                                 Periodicity.Quarterly,
                                                 yc.Scenario,
                                                 ImportFormats.Cashflow),
                                    DataNode = dn.SystemName}).ToArrayAsync()
                    .Where(x => x.DataNodes.Any())
                    .OrderBy(y => y.Year).ThenBy(y => y.Month).ToArrayAsync();*/

In [ ]:
new ImportArgs ("CH", 1, 1, Periodicity.Quarterly, null, ImportFormats.Cashflow)

In [ ]:
reportingNodeByPartitionAndScenario[(dataNodes.First().Partition, null)]